In [1]:
import numpy as np
import matplotlib.pyplot as plt


#### Defining objects(spherical) and intersection point to any given sphere

In [2]:
#Spherical objects

objects = [
    {'centre' : np.array([-0.2,0,-1]), 'radius' : 0.7, 'ambient' : np.array([0.0,0.0,0.1]), 'diffuse' : np.array([0.0,0.0,0.8]), 'specular' : np.array([1,1,1]), 'shininess' : 100, 'reflection' : 0.25},
    {'centre' : np.array([0.1,-0.3,0]), 'radius' : 0.1, 'ambient' : np.array([0.1,0.0,0.0]), 'diffuse' : np.array([0.8,0.0,0.0]), 'specular' : np.array([1,1,1]), 'shininess' : 100, 'reflection' : 0.25},
    {'centre' : np.array([-0.3,0,0]), 'radius' : 0.15, 'ambient' : np.array([0.0,0.2,0.0]), 'diffuse' : np.array([0.0,0.7,0.0]), 'specular' : np.array([1,1,1]), 'shininess' : 100, 'reflection' : 0.25},
    {'centre' : np.array([0,-9000,0]), 'radius' : 9000-0.7, 'ambient': np.array([0.1, 0.1, 0.1]), 'diffuse': np.array([0.6, 0.6, 0.6]), 'specular': np.array([1, 1, 1]), 'shininess': 100, 'reflection' : 0.25}
    
]

lights = [{'position' : np.array([7,5,5]), 'ambient' : np.array([1,1,1]), 'diffuse' : np.array([1,1,1]), 'specular' : np.array([1,1,1])},
    #{'position' : np.array([-10,10,10]), 'ambient' : np.array([1,1,1]), 'diffuse' : np.array([1,1,1]), 'specular' : np.array([1,1,1])}
]


def sphere_intersect(centre, radius, ray_origin, ray_dir):
    b = 2*np.dot(ray_dir,ray_origin-centre)
    c = np.linalg.norm(ray_origin-centre)**2 - radius**2
    a = 1
    disc = b**2-4*c
    
    if disc>0:
        t1 = (-b + np.sqrt(disc))/2
        t2 = (-b - np.sqrt(disc))/2
        if t1>0 and t2>0: # If sphere is completely behind the screen, with camera in front of the screen, then  for sure both t1 and t2 will be positive
            return min(t1,t2) # Nearest point intersection
        
    return None

#### Finding the intersection point to the nearest object in case the ray intersects any object in the scene

In [3]:
def nearest_object_intersection(objects, ray_origin, ray_dir):
    dists = [sphere_intersect(obj['centre'],obj['radius'],ray_origin,ray_dir) for obj in objects]
    nearest_obj = None
    min_t = np.inf
    for i,d in enumerate(dists):
        if d is not None and min_t>d:
            min_t = d
            nearest_obj = objects[i]
    
    return nearest_obj, min_t

#### Vector manipulation functions

In [4]:
def normalize(vector):
    return vector/np.linalg.norm(vector)

def reflected(vector, axis):
    return vector - 2*np.dot(vector,axis)*axis

### Implementing the steps

In [18]:
width = 300
height = 300
max_depth = 3

ratio = float(width/height)

camera = np.array([0,0,1])
screen = (-1,1/ratio,1,-1/ratio)

image = np.zeros((height,width,3))

#Enumerating over the screen, pixel by pixel
for i,y in enumerate(np.linspace(screen[1],screen[3],height)):
    for j,x in enumerate(np.linspace(screen[0],screen[2],width)):
        
        pixel = np.array([x,y,0])
        #Ray defined using origin and direction in the parametric equation
        origin = camera
        direction = normalize(pixel-origin)
        colour = np.zeros((3))
        reflection = 1
        
        for r in range(max_depth):
            
            #Finding the nearest object of intersection, if any
            nearest_obj, t = nearest_object_intersection(objects, origin,direction)
            if nearest_obj is None:
                break
            intersect_point = origin + direction*t

            #Initialise illumination
            illumination = np.zeros((3))

            #Checking if light source is not blocked towards this point
            #Shifting a little in the direction of the normal to avoid including the nearest sphere as an obstacle
            sphere_normal = normalize(intersect_point - nearest_obj['centre'])
            shift = intersect_point + 1e-5*sphere_normal
            for light in lights:

                obstacle_direction = normalize(light['position'] - shift)
                _,obs_t = nearest_object_intersection(objects,shift,obstacle_direction)
                light_dist = np.linalg.norm(light['position']-shift)
                is_shadowed = light_dist>obs_t
                if is_shadowed:
                    continue


                #ambiant component
                illumination += nearest_obj['ambient']*light['ambient']

                #diffuse component
                illumination += nearest_obj['diffuse']*light['diffuse']*np.dot(obstacle_direction, sphere_normal)

                #specular component
                #illumination += nearest_obj['specular']*light['specular']*(np.dot(sphere_normal,normalize(obstacle_direction-direction)))** (nearest_obj['shininess'] / 4)
                intersection_to_camera = normalize(camera - intersect_point)
                H = normalize(obstacle_direction + intersection_to_camera)
                illumination += nearest_obj['specular'] * light['specular'] * np.dot(sphere_normal, H) ** (nearest_obj['shininess'] / 4)
                
            colour += illumination*reflection
            reflection *= nearest_obj['reflection']
            origin = intersect_point
            direction = reflected(direction,sphere_normal)
            

        #Set the illumination
        image[i, j] = np.clip(colour, 0, 1)

        
        

plt.imsave('image.png',image)